## Playing with The Swish- New Activation Function for building Deep Neural Network
https://www.linkedin.com/pulse/playing-swish-new-activation-building-deep-neural-network-kumar/

In [ ]:
import numpy as np
import pandas as pd
import timeit
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
# reset tensorflow graph
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)

In [ ]:
# Data
trainX = mnist.train.images
trainY = mnist.train.labels
# Splitting Dataset
trainX, testX, trainY, testY = train_test_split(trainX, trainY, test_size=0.20, random_state=42)

In [ ]:
# Defining Parameters
logs_path = './session/v16'
save_path = logs_path + '/'
learning_rate = 0.003
n_features = 784
n_classes = 10
batch_size = 100
training_epochs = 160

In [ ]:
# Defining Hidden Layers
K = 400
L = 300
M = 200
N = 100
O = 50

In [ ]:
def swish(x):
    return x * tf.nn.sigmoid(x)

In [ ]:
## Building Network
with tf.name_scope("input"):
    # None -> batch size can be any size, with n_features
    x = tf.placeholder(tf.float32, shape=[None, n_features], name="x-input")
    # target n_classes output classes
    y_ = tf.placeholder(tf.float32, shape=[None, n_classes], name="y-input")    

In [ ]:
with tf.name_scope("Layer1"):
    W1 = tf.Variable(tf.truncated_normal([n_features, K], stddev=0.1))
    b1 = tf.Variable(tf.ones([K]))
    Y1 = swish(tf.add(tf.matmul(x, W1), b1))    

In [ ]:
with tf.name_scope("Layer2"):
    W2 = tf.Variable(tf.truncated_normal([K, L], stddev=0.1))
    b2 = tf.Variable(tf.ones([L]))
    Y2 = swish(tf.add(tf.matmul(Y1, W2), b2))    

In [ ]:
with tf.name_scope("Layer3"):
    W3 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
    b3 = tf.Variable(tf.ones([M]))
    Y3 = swish(tf.add(tf.matmul(Y2, W3), b3))    

In [ ]:
with tf.name_scope("Layer4"):
    W4 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
    b4 = tf.Variable(tf.ones([N]))
    Y4 = swish(tf.add(tf.matmul(Y3, W4), b4))    

In [ ]:
with tf.name_scope("Layer5"):
    W5 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
    b5 = tf.Variable(tf.ones([O]))
    Y5 = swish(tf.add(tf.matmul(Y4, W5), b5))    

In [ ]:
with tf.name_scope("Output"):
    # y is our prediction
    W = tf.Variable(tf.truncated_normal([O, n_classes], stddev=0.1))
    b = tf.Variable(tf.ones([n_classes]))
    Ylogits = tf.matmul(Y5, W) + b
    y = tf.nn.softmax(Ylogits)    

In [ ]:
# Defining cost function
with tf.name_scope("Loss"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=y_)
    cross_entropy = tf.reduce_mean(cross_entropy) * 100    

In [ ]:
# Using AdadeltaOptimizer
with tf.name_scope("train"):
    learn_rate = tf.Variable(learning_rate)
    train_op = tf.train.AdadeltaOptimizer(learn_rate).minimize(cross_entropy)    

In [ ]:
with tf.name_scope("Accuracy"):
    prediction = tf.argmax(y, 1, name="Predict")
    # Accuracy
    correct_prediction = tf.equal(prediction, tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")    

In [ ]:
# create a summary for our cost and accuracy
training_loss = tf.summary.scalar("training_loss", cross_entropy)
training_accuracy = tf.summary.scalar("training_accuracy", accuracy)
test_loss = tf.summary.scalar("test_loss", cross_entropy)
test_accuracy = tf.summary.scalar("test_accuracy", accuracy)

In [ ]:
# Creating a Saver to save the graph
saver = tf.train.Saver()

In [ ]:
## Timer
start = timeit.default_timer()

# Start Graph Execution
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    for epoch in range(training_epochs):
        batch_count = int(trainX.shape[0] / batch_size)
        
        for i in range(batch_count):
            batch_x = trainX[i * batch_size: i * batch_size + batch_size]
            batch_y = trainY[i * batch_size: i * batch_size + batch_size]
            
            _, acc, loss = sess.run([train_op, training_accuracy, training_loss], feed_dict={x: batch_x, y: batch_y})
            
            writer.add_summary(acc, epoch * batch_count + i)
            writer.add_summary(acc, epoch * batch_count + i)
            
        acc, loss, a_loss = sess.run([test_accuracy, test_loss, cross_entropy], feed_dict={x: testX, y_:testY})
        writer.add_summary(acc, epoch * batch_count + i)
        writer.add_summary(loss, epoch * batch_count + i)
        print("Test loss at step: ", epoch, " is ", a_loss)
        if epoch % 5 == 0:
            print("Epoch: ", epoch)
    print("Accuracy: ", accuracy.eval(feed_dict={x: testX, y_: testY}))
    saver.save(sess, save_path + "model.ckpt")
    print("Done")
    
stop = timeit.default_timer()
print(stop - start)